# Centrality Algorithms

<a target="_blank" href="https://colab.research.google.com/github/neo4j/graph-data-science-client/blob/main/examples/centrality-algorithms.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This Jupyter notebook is hosted [here](https://github.com/neo4j/graph-data-science-client/blob/main/examples/centrality-algorithms.ipynb) in the Neo4j Graph Data Science Client Github repository.

Centrality algorithms are used to understand the role or influence of particular nodes in a graph. The notebook shows the application of centrality algorithms using the `graphdatascience` library on the Airline travel reachability network dataset that can be downloaded [here](https://snap.stanford.edu/data/reachability.html).

This notebook will show how you can apply eigenvector centrality, betweenness centrality, degree centrality and closeness centrality on a graph dataset.

### Setup

We start by importing our dependencies and setting up our GDS client connection to the database.

In [2]:
# Install necessary dependencies
#%pip install graphdatascience pandas

In [1]:
from graphdatascience import GraphDataScience
import pandas as pd
import os

C:\Users\kedar\anaconda3\envs\graph_stuff\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
# NEO4J_AUTH = None
# if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
#     NEO4J_AUTH = (
#         os.environ.get("NEO4J_USER"),
#         os.environ.get("NEO4J_PASSWORD"),
#     )

# gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

# # Replace with the actual connection URI and credentials
NEO4J_CONNECTION_URI = "bolt://3.235.129.183:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "incentives-sevenths-pear"

# Client instantiation
gds = GraphDataScience(
    NEO4J_CONNECTION_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)


### Importing the dataset

We import the dataset as a pandas dataframe first. We deal with two files here. The file `reachability-meta.csv.gz` stores the names of the cities and their information while the file `reachability.txt.gz` stores the edges of the graph. An edge exists from city `i` to city `j` if the estimated airline travel time is less than a threshold.


In [3]:
nodes_info_df = pd.read_csv('https://snap.stanford.edu/data/reachability-meta.csv.gz', compression='gzip')
nodes_info_df.head()

,node_id,name,metro_pop,latitude,longitude
0,0,"Abbotsford, BC",133497.0,49.051575,-122.328849
1,1,"Aberdeen, SD",40878.0,45.459090,-98.487324
2,2,"Abilene, TX",166416.0,32.449175,-99.741424
3,3,"Akron/Canton, OH",701456.0,40.797810,-81.371567
4,4,"Alamosa, CO",9433.0,37.468180,-105.873599


In [4]:
routes_df = pd.read_csv('https://snap.stanford.edu/data/reachability.txt.gz', sep=' ', skiprows=6, header=None, compression='gzip', names=['Origin', 'Destination', 'Weight'])
routes_df.head()

,Origin,Destination,Weight
0,27,0,-757
1,57,0,-84
2,70,0,-1290
3,74,0,-465
4,86,0,-700


Next, we load this data (nodes and edges) into a Graph Database and a GDS graph.

In [5]:
gds.run_cypher(
    "UNWIND $nodes AS node CREATE (n:City {node_id: node.node_id, name: node.name, population: node.metro_pop})",
    params={"nodes": nodes_info_df.to_dict("records")},
)

gds.run_cypher(
    """
    UNWIND $rels AS rel 
    MATCH (source:City {node_id: rel.Origin}), (target:City {node_id: rel.Destination}) 
    CREATE (source)-[:HAS_FLIGHT_TO]->(target)
    """,
    params={"rels": routes_df.to_dict("records")},
)

""


In [6]:
G, result = gds.graph.project("airline", "City", "HAS_FLIGHT_TO")

print(f"The projection took {result['projectMillis']} ms")

# We can use convenience methods on `G` to check if the projection looks correct
print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")

The projection took 214 ms
Graph 'airline' node count: 456
Graph 'airline' node labels: ['City']


### Eigenvector Centrality

[Eigenvector centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/) measures the importance or influence of a node based on its connections to other nodes in the network. A higher eigenvector centrality score suggests that a node is more central and influential within the network.

For our dataset, eigenvector centrality can help identify airports that are not only well-connected themselves but also have connections to other important airports. Nodes with high eigenvector centrality are likely to be major hubs or airports with extensive connectivity.

In [7]:
eigenvector_centrality_result = gds.eigenvector.mutate(G, maxIterations=100, mutateProperty="eigenvectorCentrality")

In [11]:
# We can verify that the eigenvectorCentrality was mutated
G.node_properties()

City    [eigenvectorCentrality]
dtype: object

We can see if our implementation converged or not and if converged, the number of iterations it took using the below code:

In [13]:
if eigenvector_centrality_result.didConverge: 
    print(f"The number of iterations taken by Eigenvector Centrality to run is {eigenvector_centrality_result.ranIterations}.")
else:
    print("Algorithm did not converge!")

The number of iterations taken by Eigenvector Centrality to run is 13.


We can also see the distribution of the eigenvector centrality measures using the below code. This will show us the minimum, maximum, mean and other statistical values for our centrality measure.

In [8]:
eigenvector_centrality_result.centralityDistribution

{'p99': 0.08469437807798386,
 'min': 0.0012630745768547058,
 'max': 0.0856785699725151,
 'mean': 0.041094380038741385,
 'p90': 0.07575749605894089,
 'p50': 0.0386042520403862,
 'p999': 0.0856785699725151,
 'p95': 0.08167456835508347,
 'p75': 0.05822562426328659}

In [10]:
gds.graph.nodeProperties.write(G, ["eigenvectorCentrality"])

writeMillis                              405
graphName                            airline
nodeProperties       [eigenvectorCentrality]
propertiesWritten                        456
Name: 0, dtype: object

Using the results from eigenvector centrality, we can now look up the top 20 cities with airports that have major hubs or airports with extensive connectivity.

In [12]:
# Execute the Cypher query to retrieve the top 20 cities with eigenvector centrality
query = """
MATCH (n:City)
RETURN n.node_id AS node_id, n.name AS name, n.population AS population, n.eigenvectorCentrality AS eigenvectorCentrality
ORDER BY n.eigenvectorCentrality DESC
LIMIT 20
"""
result = gds.run_cypher(query)

# Display the result
print(result)

    node_id                     name  population  eigenvectorCentrality
0       246          Los Angeles, CA  12940000.0               0.085678
1       368        San Francisco, CA   4391000.0               0.085393
2        94    Dallas/Fort Worth, TX   6527000.0               0.085097
3       230            Las Vegas, NV   1970000.0               0.084824
4        74              Chicago, IL   9505000.0               0.084694
5       100               Denver, CO   2600000.0               0.084620
6       324              Phoenix, AZ   4263000.0               0.084607
7       383       Seattle/Tacoma, WA   3500000.0               0.084396
8       434           Washington, DC   5704000.0               0.084002
9       269  Minneapolis/St Paul, MN   3318000.0               0.083769
10      294             New York, NY  19020000.0               0.083696
11      323         Philadelphia, PA   5992000.0               0.083678
12      102              Detroit, MI   4286000.0               0

### Betweenness Centrality

[Betweenness Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/betweenness-centrality/) quantifies the importance of a node as a bridge or intermediary in the network. It measures how often a node lies on the shortest path between other pairs of nodes. 

For our dataset, cities/airports with high betweenness centrality serve as crucial transfer points or connecting hubs between airports that might not have direct flights between them. They play a significant role in facilitating the flow of air travel and can be vital for overall network connectivity.

In [14]:
betweenness_centrality_result = gds.betweenness.mutate(G, mutateProperty="betweennessCentrality")

Failed to write data to connection IPv4Address(('3.235.129.183', 7687)) (ResolvedIPv4Address(('3.235.129.183', 7687)))
Failed to write data to connection IPv4Address(('3.235.129.183', 7687)) (ResolvedIPv4Address(('3.235.129.183', 7687)))
BetweennessCentrality: 100%|██████████| 100.0/100 [00:04<00:00, 22.62%/s]


In [15]:
# We can verify that the betweennessCentrality was mutated
G.node_properties()

City    [betweennessCentrality, eigenvectorCentrality]
dtype: object

We can also see the distribution of the betweenness centrality measures using the below code. This will show us the minimum, maximum, mean and other statistical values for our centrality measure.

In [16]:
betweenness_centrality_result.centralityDistribution

{'p99': 3265.2968747615814,
 'min': 0.04446244239807129,
 'max': 3628.7656247615814,
 'mean': 298.0898581821668,
 'p90': 1006.8867185115814,
 'p50': 21.20861792564392,
 'p999': 3628.7656247615814,
 'p95': 1823.7578122615814,
 'p75': 184.29199194908142}

In [17]:
gds.graph.nodeProperties.write(G, ["betweennessCentrality"])

writeMillis                               95
graphName                            airline
nodeProperties       [betweennessCentrality]
propertiesWritten                        456
Name: 0, dtype: object

Using the results from betweenness centrality, we can now look up the top 20 cities with airports that serve as crucial transfer points or connecting hubs between airports that might not have direct flights between them.

In [18]:
# Execute the Cypher query to retrieve the top 20 cities with betweenness centrality
query = """
MATCH (n:City)
RETURN n.node_id AS node_id, n.name AS name, n.population AS population, n.betweennessCentrality AS betweennessCentrality
ORDER BY n.betweennessCentrality DESC
LIMIT 20
"""
result = gds.run_cypher(query)

# Display the result
print(result)

    node_id                     name  population  betweennessCentrality
0       246          Los Angeles, CA  12940000.0            3628.755650
1       100               Denver, CO   2600000.0            3435.693657
2       294             New York, NY  19020000.0            3297.750685
3       416              Toronto, ON   6324000.0            3295.316691
4       368        San Francisco, CA   4391000.0            3265.288064
5        74              Chicago, IL   9505000.0            3250.731388
6       230            Las Vegas, NV   1970000.0            3156.983626
7       434           Washington, DC   5704000.0            3108.796793
8        94    Dallas/Fort Worth, TX   6527000.0            3094.760797
9       324              Phoenix, AZ   4263000.0            2815.294080
10      383       Seattle/Tacoma, WA   3500000.0            2600.388338
11      269  Minneapolis/St Paul, MN   3318000.0            2386.544069
12      367            San Diego, CA   3140000.0            2225

In [ ]:
### Betweenness Centrality

[Betweenness Centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/betweenness-centrality/) quantifies the importance of a node as a bridge or intermediary in the network. It measures how often a node lies on the shortest path between other pairs of nodes. 

For our dataset, cities/airports with high betweenness centrality serve as crucial transfer points or connecting hubs between airports that might not have direct flights between them. They play a significant role in facilitating the flow of air travel and can be vital for overall network connectivity.

### Cleanup

Before finishing we can clean up the example data from both the GDS in-memory state and the database.

In [26]:
# Cleanup GDS
G.drop()

graphName                                                          airline
database                                                             neo4j
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                              456
relationshipCount                                                    71959
configuration            {'relationshipProjection': {'HAS_FLIGHT_TO': {...
density                                                           0.346824
creationTime                           2023-05-31T16:40:51.805849930+00:00
modificationTime                       2023-05-31T16:58:18.406045233+00:00
schema                   {'graphProperties': {}, 'relationships': {'HAS...
schemaWithOrientation    {'graphProperties': {}, 'relationships': {'HAS...
Name: 0, dtype: object

In [27]:
# Cleanup database
gds.run_cypher("MATCH (n:City) DETACH DELETE n")

""


### References
- For the network:
Brendan J. Frey and Delbert Dueck. "Clustering by passing messages between data points." Science 315.5814 (2007): 972-976.

- For the city metadata (metropolitan population, latitude, and longitude):
Austin R. Benson, David F. Gleich, and Jure Leskovec. "Higher-order Organization of Complex Networks." Science, 353.6295 (2016): 163–166.

- Notebook contributed by [Kedar Ghule](https://github.com/kedarghule)